In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
!python3 -m spacy download en_core_web_sm

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a30e6907-8142-4b12-9465-54dd96ae8823/bin/python3: No module named spacy


In [0]:
import nltk, re, math
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Out[2]: True

In [0]:
# 1. Get plot summaries, movie data, and search queries into rdd format
plot_summaries = sc.textFile("dbfs:/FileStore/tables/plot_summaries.txt")
movie_metadata = sc.textFile("dbfs:/FileStore/tables/movie_metadata.tsv")
search_queries = sc.textFile("dbfs:/FileStore/tables/search_queries-3.txt")

In [0]:
# 2. Obtain information and remove stopwords from summaries
stop_words = set(stopwords.words('english'))
# Obtain plot summaries in lowercase without any punctuation and with movieID as a key and terms as values
plot_summ_rdd = plot_summaries.map(lambda x: re.sub(r'[^\w\s]', '', x.lower())).map(lambda x: (x.split("\t")[0], x.split("\t")[1]))
# Recombine the words in the plot summary as one big text after splitting and removing stopwords and empty words
plot_summ_rdd2 = plot_summ_rdd.map(lambda x: (x[0], x[1].split(" "))).map(lambda x: (x[0], ' '.join([word for word in x[1] if word not in stop_words and word != ''])))

In [0]:
# Unique movieIds obtained
collection_docs = set(plot_summ_rdd2.keys().collect())
# Number of documents
N = len(collection_docs)
# List format of unique documents / movieIds
all_docs_list = list(collection_docs)


In [0]:
# Make an rdd where movieID is the key and the value is a list of all terms, then map it such that the rdd is a collection of (movieID, term) pairs.
terms_rdd = plot_summ_rdd2.map(lambda x: (x[0], x[1].split(" "))).map(lambda x: [(x[0], word) for word in x[1]]).flatMap(lambda x: x)

# Term frequencies dictionary produced by counting by value where the value is currently the term.
term_freq_dict = terms_rdd.countByValue()


In [0]:
# Rearrange the term frequency dictionary to have key as (term, movieID) and value as frequency
tf_ij = sc.parallelize(term_freq_dict.items()).map(lambda x: ((x[0][1], x[0][0]), x[1]))
# Group by key gets you all the potential documents / movie IDs that are involved with the term, find the number of unique documents per key and you get # documents that term is in.
# Using map, make an rdd where the term is the key and the movieID is the value. 
# Group by key to get every document the key/term is in in a list format, find the length of unique values of that list to find the number of documents the term is in.
term_doc_count = terms_rdd.map(lambda x: (x[1], x[0])).groupByKey().map(lambda x : (x[0], len(list(set(x[1])))))


In [0]:
# 3. Create an RDD to track tf_idf values for every term, movieID pair.
# We first map to force a term as our key and the rest of the data as a value so we can join with the RDD that tracks the number of documents a term is in, term_docs_count
# Then, we map the result to have key-value pairs of term and movieID, with the value being a tuple containing both the term frequency and the term_doc_count value
# Do one final map to calculate the tf_idf from the corresponding values.
tf_idf = tf_ij.map(lambda x: (x[0][0],(x[0][1], x[1]))).join(term_doc_count).map(lambda x: ((x[0], x[1][0][0]), (x[1][0][1], x[1][1]))).map(lambda x: (x[0], x[1][0] * math.log10(N/x[1][1])))
tf_idf.collect()

Out[9]: [(('shlykov', '23890098'), 4.626401965060686),
 (('wealth', '31186339'), 1.897237175367916),
 (('wealth', '1591973'), 1.897237175367916),
 (('wealth', '20903293'), 1.897237175367916),
 (('wealth', '34501845'), 1.897237175367916),
 (('wealth', '8423977'), 1.897237175367916),
 (('wealth', '1740764'), 3.794474350735832),
 (('wealth', '27923824'), 1.897237175367916),
 (('wealth', '26449950'), 1.897237175367916),
 (('wealth', '2648856'), 3.794474350735832),
 (('wealth', '41528'), 1.897237175367916),
 (('wealth', '22039671'), 1.897237175367916),
 (('wealth', '26039454'), 1.897237175367916),
 (('wealth', '6763625'), 1.897237175367916),
 (('wealth', '18900457'), 1.897237175367916),
 (('wealth', '1430396'), 1.897237175367916),
 (('wealth', '12562570'), 1.897237175367916),
 (('wealth', '2780906'), 1.897237175367916),
 (('wealth', '35574760'), 1.897237175367916),
 (('wealth', '1374349'), 3.794474350735832),
 (('wealth', '15441884'), 1.897237175367916),
 (('wealth', '6012592'), 1.897237175

In [0]:
# 4. Analyze user input
# First, obtain search queries and break them down into single term entries or multi-term entries
user_search_list = search_queries.collect()
single_term_results = []
multi_term_results = []
for query in user_search_list:
    if len(query.split(" ")) == 1:
        # Only one term in this line
        # Get the ones where the term matches the query and get the top 10 movies based on the tf-idf value
        single_term_results.append(tf_idf.filter(lambda x: x[0][0] == query).sortBy(lambda x: -x[1]).take(10))
    else:
        # Multiple terms in this line
        multi_term_results.append(query)


In [0]:
# 4a. Output the top 10 documents with the highest tf-idf values for singular terms in the search_queries file.
# Prep movie metadata information
# Get an RDD with (movieID, movie name) tuples
movie_meta_pair = movie_metadata.map(lambda x: (x.split("\t")[0], x.split("\t")[2]))
# For every single term query
for result in single_term_results:
    # Keep track of which movie is best via a counter value
    count = 10
    # Store result in movies_results list where found_movie[0][1] is the movieID
    movies_results = []
    for found_movie in result:
        movies_results.append((found_movie[0][1], count))
        count -= 1
    # Parallelize the list of (movieID, rank) tuples
    movies_rdd = sc.parallelize(movies_results)
    
    # Join the movies_rdd with the movie_meta_pair rdd on the key and sort it based off of the rank from movies_rdd. Collect only the movie Names
    res_movies = movies_rdd.join(movie_meta_pair).sortBy(lambda x: -x[1][0]).map(lambda x: x[1][1]).collect()
    # Print the single term  along with the list of movies that go with the term
    print(result[0][0][0], "\t", res_movies)

wealth 	 ['Saraswathi Sabatham', 'Tiga Abdul', 'Beta', 'His Highness Abdullah', 'Alexander The Great', 'Ahmad Albab', "The Ghost Valley's Treasure Mysteries", 'Intolerable Cruelty', 'Heart of Stone', 'God of Gamblers']
tycoon 	 ['Oily Hare', 'Shikari', 'Dream Lover', 'Takkari', 'Dil Hai Ke Manta Nahin', 'Dubai', 'Motives', 'Daffy Dilly', 'Frivolous Wives', 'Machakaaran']
shlykov 	 ['Taxi Blues']


In [0]:
# Function to calculate cos similarity between two vectors represented as dictionaries with (term, tf-idf) pairs
def cos_sim(d1, d2):
    # Calculate dot product by getting, for each term, the tf-idf value and multiplying across vectors
    dot_prod = 0
    for key in d1.keys():
        dot_prod += d1[key] * d2[key]
    # For all tf-idf values, square them and sum them, then find the sqrt of the sum
    norm_d1 = math.sqrt(sum(list(map(lambda x: x * x, d1.values()))))
    norm_d2 = math.sqrt(sum(list(map(lambda x: x * x, d2.values()))))
    # Account for division by 0
    if norm_d1 == 0 or norm_d2 == 0:
        return 0
    # Return cos similarity measure
    return (float(dot_prod) / (float(norm_d1) * float(norm_d2)))

In [0]:
all_terms = term_doc_count.keys().collect()
for result in multi_term_results:
    all_query_terms = result.split(" ")
    # Make a vector to represent the query's tf-idf
    query_vector = []
    doc_vectors = dict()
    # Keep track of the frequency of terms within the query itself
    query_term_freq = dict()
    for term in all_query_terms:
        query_term_freq.update({term: query_term_freq.get(term, 0) + 1})
    simple_rdd = sc.parallelize([])
    #print(simple_rdd.collect())
    # Split up the query into terms so that we can calculate each term's IDF, then the term's tf_idf
    for term in all_query_terms:
        if term in all_terms:
            # term_DF represents the number of documents the term is in, just like finding the DF for each term in general.
            term_DF = term_doc_count.filter(lambda x: x[0] == term).collect()[0][1]
            # IDF is calculated as specified with the log of (N / Ni) where i is a term
            term_IDF = math.log10(N/term_DF)
            # The TF-IDF calculation uses the dictionary we made to track term frequencies to multiply with the IDF for that term in this document
            term_tf_idf = query_term_freq.get(term) * term_IDF
            # Append each (term, tf-idf) pair to the vector.
            query_vector.append((term, term_tf_idf))
            simple_rdd = simple_rdd.union(tf_idf.filter(lambda x: x[0][0] == term))
        else:
            # tf-idf for that term is 0
            query_vector.append((term, 0))
    
    # An RDD of (movieID, dictionary of term: tf-idf pairs)
    doc_specific_dict = simple_rdd.map(lambda x: (x[0][1], ({x[0][0]: x[1]}))).groupByKey().map(lambda x : (x[0], {key: val for dic in x[1] for key, val in dic.items()})).collectAsMap()
    # Represent query vector as a dict
    query_vector_dict = sc.parallelize(query_vector).collectAsMap()
    
    # For every document, make a document vector and calculate the cos similarity between the query and the document
    for doc in all_docs_list:
        doc_vector = dict()
        for term in all_query_terms:
            doc_vector[term] = doc_specific_dict.get(doc, dict()).get(term, 0)
        doc_vectors[doc] = cos_sim(query_vector_dict, doc_vector)
    # Sort and obtain movie titles for top movies
    top_movies = sorted(doc_vectors.items(), key=lambda x: -x[1])[1:10]
    top_movies_rdd = sc.parallelize(top_movies)
    res_movies = top_movies_rdd.join(movie_meta_pair).sortBy(lambda x: -x[1][0]).map(lambda x: x[1][1]).collect()
    # Print results as well as query provided by user
    print(result, "\t", res_movies)

death commit suicide 	 ['Sublime', 'Ned Kelly', 'Le Dernier cri', 'Manmadhan', 'Let George Do It', 'Life with Judy Garland: Me and My Shadows', 'Little Shop of Horrors', 'The Grey', 'Magnificent Butcher']
commit suicide by 	 ['Conviction', 'Aatmakatha', 'Dr. Dolittle', 'Merupu Daadi', 'Manmadhan', 'The Air I Breathe', 'Dhoom', 'Life with Judy Garland: Me and My Shadows', 'Little Shop of Horrors']
tale man quest 	 ['The River Why', 'Champion', '90 Miles', 'The Forbidden Kingdom', 'The Adventures of Baron Munchausen', 'Here Comes Peter Cottontail', 'The Four Feathers', 'So Dear to My Heart', "Rudolph's Shiny New Year"]
two sisters 	 ['Star Trek: Generations', 'Red Dirt', 'Gummo', 'Lies', 'Pals and Gals', 'Passport to Paris', 'The Juniper Tree', 'That Darn Cat!', 'The Snapper']
